<a href="https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/dsportfolio/Coronavirus_dataset_enrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Model Training](https://drive.google.com/uc?id=17cUxpbL2eonLzgY0uIVAxmsfMfQF1by2)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls '/content/drive/My Drive/Colab Notebooks/twitter.properties'

In [0]:
!pip install ConfigParser

[twitter]
accesstoken=xxxx

accesstokensecret=xxxx

apikey=xxxxxx

apisecretkey=xxxxxx

googleapikey=xxxxxxx

In [0]:
import pandas as pd
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitter.properties')

print(config.sections());


In [0]:
corona_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')

In [0]:
corona_confirmed_df.head()

In [0]:
corona_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

In [0]:
corona_death_df.head()

In [0]:
corona_recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [0]:
corona_recovered_df.head()

In [0]:
corona_confirmed_df=corona_confirmed_df.melt(id_vars=['Province/State','Country/Region','Lat','Long'])

In [0]:
corona_confirmed_df.head()

In [0]:
corona_confirmed_df=corona_confirmed_df.rename({'variable':'Date','value': 'Confirmed'}, axis='columns')

In [0]:
corona_confirmed_df.head()

In [0]:
corona_death_df=corona_death_df.melt(id_vars=['Province/State','Country/Region','Lat','Long'])

In [0]:
corona_death_df=corona_death_df.rename({'variable':'Date','value': 'Death'}, axis='columns')

In [0]:
corona_death_df.head()

In [0]:
corona_recovered_df=corona_recovered_df.melt(id_vars=['Province/State','Country/Region','Lat','Long']).rename({'variable':'Date','value': 'Recovered'}, axis='columns')

In [0]:
corona_recovered_df.head()

In [0]:
combined_df = [corona_confirmed_df, corona_death_df, corona_recovered_df]
combined_df = [df.set_index(['Province/State','Country/Region','Lat','Long','Date']) for df in combined_df]
combined_df=combined_df[0].join(combined_df[1:])

In [0]:
combined_df.head()

In [0]:
combined_df=combined_df.reset_index()

In [0]:
combined_df.head()

In [0]:
combined_df[['Lat', 'Long','Confirmed','Death','Recovered']] = combined_df[['Lat', 'Long','Confirmed','Death','Recovered']].apply(pd.to_numeric) 

In [0]:
combined_df[['Date']] = combined_df[['Date']].apply(pd.to_datetime)

In [0]:
combined_df.dtypes

In [0]:
combined_df.head()

In [0]:

combined_df=combined_df.rename({'Province/State':'State','Country/Region': 'Country'}, axis='columns')

In [0]:
combined_df.head()

In [0]:
combined_df.isnull().any()

In [0]:
combined_df.count()

In [0]:
combined_df.set_index('Country').filter(like="India", axis=0)

In [0]:
combined_df[combined_df.State.isnull()]['Country'].value_counts()

In [0]:
!pip install -U googlemaps

In [0]:
import googlemaps

In [0]:
gmaps = googlemaps.Client(key=config.get('twitter', 'googleapikey'))

geocode_result = gmaps.reverse_geocode((-23.4425, -58.4438))

print(geocode_result[0])
print(geocode_result[0]['formatted_address'])
print(geocode_result[0]['formatted_address'].split(",")[-2].strip(" "))

In [0]:
import re

english_check = re.compile(r'[a-z]')

def get_state(lat, longi):
  try:
    output=gmaps.reverse_geocode((lat, longi))[0]['formatted_address'].split(",")[-2].strip()
    if(len(output.split(" ")) > 1):
      output="TBF"

    if not english_check.match(output.lower()):
      output="TBF"
    
  except:
    print("Error in Reverse Geocoding")
    output="TBF"
  return output

In [0]:
combined_df["state_cleaned"]=combined_df[combined_df.State.isnull()][['Lat','Long']].apply(lambda x : get_state(x['Lat'], x['Long']),axis=1)


In [0]:
combined_df[combined_df.state_cleaned.isnull()]['Country'].value_counts()

In [0]:
combined_int_df = combined_df.copy()

In [0]:
combined_df[combined_df.state_cleaned.notna()]

In [0]:
combined_int_df.count()

In [0]:
combined_int_df[combined_int_df.State.notna()]

In [0]:
combined_int_df[combined_int_df.State.notna()]["State"].count()

In [0]:
import numpy
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: x['State'] if (numpy.all(pd.notnull(x['State']))) else x['state_cleaned'], axis=1)

In [0]:
combined_int_df[combined_int_df.State.notna()]

In [0]:
combined_int_df[combined_df.state_cleaned=="TBF"]

In [0]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [0]:
combined_int_df.head(50)

In [0]:
capital_df = pd.read_csv('https://raw.githubusercontent.com/icyrockcom/country-capitals/master/data/country-list.csv')

In [0]:
capital_df.head()

In [0]:
capital_df['country'].replace({"United Kingdom; England": "United Kingdom"}, inplace=True)

In [0]:
capital_df['country']=capital_df.country.str.lower()

In [0]:
capital_df.head()

In [0]:
capital_df=capital_df.set_index('country')

In [0]:
capital_df.loc['algeria'][0]

In [0]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]

In [0]:
def lookup_capital(country):
  try:
    capital=capital_df.loc[country.lower()][0]
  except:
    capital="TBF"
  return capital  

In [0]:
combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [0]:
combined_int_df['state_cleaned']=combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [0]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [0]:
combined_int_df["Country"].replace({"Republic of Moldova": "Moldova", "Holy See": "Vatican City", "UK": "United Kingdom","Viet Nam":"Vietnam","Iran (Islamic Republic of)":"Iran"}, inplace=True)

In [0]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [0]:
combined_int_df['state_cleaned']=combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [0]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [0]:
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: x['Country'] if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [0]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [0]:
combined_int_df.to_csv('corona_dataset.csv')

In [0]:
combined_int_df[combined_int_df.Country=="US"]

In [0]:
usstates_df = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')

In [0]:
usstates_df.head()

In [0]:
usstates_df=usstates_df.set_index('Abbreviation')

In [0]:
usstates_df.head()

In [0]:
usstates_df.loc['AZ']

In [0]:
combined_int_df['City']=combined_int_df.apply(lambda x: x['State'].split(",")[0] if(x['Country']=="US") else "", axis=1)

In [0]:
def convert_state(state):
  try:
    output=state.split(",")[1].strip()
    output=usstates_df.loc[output.upper()][0]
  except:
    output=state
  return output  

In [0]:
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: convert_state(x['state_cleaned']) if(x['Country']=="US") else x['state_cleaned'], axis=1)

In [0]:
combined_int_df.head(50)

In [0]:
combined_int_df[combined_int_df.Country=="US"][0:100]

In [0]:
combined_int_df.to_csv("corona_dataset.csv")

In [0]:
combined_int_df.apply(lambda x: 1 if(x['Country']=="US" and numpy.all(pd.notnull(x['City']))) else 0, axis=1)